In [9]:
from InitialGraphs import Graphdata
from InitialPattern2 import Pattern
from GraphPattern import PatternMarge
import NPGSAlgorithm as na
import time
import os
import math

fileName = 'ENZYMES'
pers = 20 # N-hop Pattern Filtering Ratio
psize = 100000 # 𝐹𝑟𝑒𝑞𝑢𝑒𝑛𝑡 𝑃𝑎𝑡𝑡𝑒𝑟𝑛 𝑀𝑎𝑛𝑎𝑔𝑚𝑒𝑛𝑡 Table Size
batchsize = 1
windowsize = 1
DxMtrix = {}
ExceptionList = {}
iterations = 0
iterations2 = 0
FPMT = {}
th2 = batchsize * windowsize
avg_time = 0

def Top_nPercent(dict, n):
    if len(dict) == 0:
        return 0
    part_v = (len(dict)-1)*n//100
    value = list(sorted(dict.items(), key = lambda item: item[1], reverse = True))[part_v][1]
    return value

def convert_size(size_bytes):
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    return "%s %s" % (s, size_name[i])

s2 = time.time()
for i in range(1):
    s1 = time.time()
    # IGPath = './Gen_Evolution_set/' + fileName + '/' + fileName + '_' + str(i) + '.edges'
    # lbPath = './dataset/' + fileName + '/' + fileName + '.node_labels'
    IGPath = './dataset/' + fileName + '/' + fileName + '.edges' #sys.argv[1]
    lbPath = './dataset/' + fileName + '/' + fileName + '.node_labels' #sys.argv[2]
    # IGPath = './dataset/Amazon/Amazon0302.txt'
    # lbPath = './dataset/Amazon/amazon.node_labels'
    # Separator = '	'
    Separator = ','

    DataT0 = Graphdata(IGPath, lbPath, Separator)
    
    ohpValue = list(set(DataT0.OhP.values()))
    ohpValue.sort(reverse=True)


    part_v = len(ohpValue)*pers//100
    Th1 = ohpValue[part_v]
    print(Th1)
    # Th1 = Top_nPercent(DataT0.OhP, pers)
    # print(len(DataT0.nList))
    # print(len(DataT0.IG.edge_list))
    iterations = i%batchsize
    
    if iterations < batchsize:
        FP = DataT0.OhP

        for ohp in FP.keys():
            ohpRe = ohp.split('/')
            ohpRe.reverse()
            ohpReverse = '/'.join(ohpRe)
            
            if ohp not in DxMtrix or ohpReverse not in DxMtrix:
                Raw = [[0 for col in range(batchsize)] for row in range(windowsize)]
                if FP[ohp] >= Th1:
                    Raw[iterations2][iterations] = 1
                    DxMtrix[ohp] = Raw
            else:
                if FP[ohp] >= Th1:
                    if ohpReverse in DxMtrix:
                        DxMtrix[ohpReverse][iterations2][iterations] = 1
                    else:
                        DxMtrix[ohp][iterations2][iterations] = 1
        
        if iterations == batchsize-1:
            for ohp in DxMtrix:
                if ohp not in FPMT:
                    Raw2 = [0 for col in range(windowsize+2)]
                    Raw2[iterations2] = sum(DxMtrix[ohp][iterations2])
                    FPMT[ohp] = Raw2
                else:
                    if i >= th2:
                        if FPMT[ohp][windowsize+1] == 0:
                            FPMT[ohp][iterations2] = sum(DxMtrix[ohp][iterations2])
                    else:
                        FPMT[ohp][iterations2] = sum(DxMtrix[ohp][iterations2])
                        
            for nhp in FPMT:
                if i >= th2:
                    if FPMT[nhp][windowsize+1] != 0:
                        FPMT[nhp][iterations2] = 0
                        FPMT[nhp][windowsize+1] = 0
                        FPMT[nhp][windowsize] = sum(FPMT[nhp][:windowsize])
                        ExceptionList[nhp] = FPMT[nhp]
                    else:
                        FPMT[nhp][windowsize] = sum(FPMT[nhp][:windowsize])
                        if FPMT[nhp][windowsize] <= int(th2*0.4):
                            FPMT[nhp][windowsize+1] = -1
                        elif FPMT[nhp][windowsize] >= int(th2*0.8):
                            FPMT[nhp][windowsize+1] = 1
                        else:
                            FPMT[nhp][windowsize+1] = 0
                else:
                    FPMT[nhp][windowsize] = sum(FPMT[nhp][:windowsize])
                    if FPMT[nhp][windowsize] <= int(th2*0.4):
                        FPMT[nhp][windowsize+1] = -1
                    elif FPMT[nhp][windowsize] >= int(th2*0.8):
                        FPMT[nhp][windowsize+1] = 1
                    else:
                        FPMT[nhp][windowsize+1] = 0

            iterations2 += 1
            
            if iterations2 == windowsize:
                iterations2 = 0
       
    updateWindow = (i+1)%th2
    delList = []
    if updateWindow == 0:
        for nhp in FPMT:
            if FPMT[nhp][windowsize] == 0:
                delList.append(nhp)
    
    for delOne in delList:
        del FPMT[delOne]
                
    if i >= th2-1:
        DataP0 = Pattern(FPMT, DxMtrix, ExceptionList, th2, psize)
        FPMT = DataP0.NhP
        ExceptionList = {}
        PM = PatternMarge(DataT0.IG, FPMT, DataT0.lbDict, DataT0.nList)
        Ge = PM.SG
        P_id_list = list(sorted(PM.pidDict.items(), key = lambda item: item[1], reverse = True))
        Gp = P_id_list
        Gl = list(sorted(PM.SGnl.items()))
        Pn = list(sorted(PM.pidn_mnDict.items(), key = lambda item: item[1], reverse = True))
        
        savefile = './Result/FPGS2/'+ fileName + '/' + fileName + '_' + str(len(FPMT)) + '_' + str(pers) + '_' + str(i)
        savedata = [Ge, Gl, Gp, Pn]
        extension = ['.edges', '.node_labels', '.pattern', '.pattern_nodes']
        
        for sd in range(len(savedata)):
            f = open(savefile + extension[sd], 'w')
            for e in savedata[sd]:
                str_e = e[0] + ',' + e[1] + '\n'
                f.write(str_e)
        f.close()

        FPMTs = len(FPMT)
        NLs = len(PM.SGnl)
        Es = len(PM.SG)
        
        file_size = 0
        for e in extension:
            file_size = file_size + os.path.getsize(savefile + e) 
            print(e.split('.')[1], convert_size(os.path.getsize(savefile + e)))
        print('OutPut File Size:', convert_size(file_size), 'bytes')

        input_file = [IGPath, lbPath]
        file_size1 = 0
        for ip in input_file:
            file_size1 = file_size1 + os.path.getsize(ip)
            print(ip.split('.')[2], convert_size(os.path.getsize(ip)))
        print('InPut File Size:', convert_size(file_size1), 'bytes')
        e1 = time.time() - s1
        print(f'Th : {Th1}, FPMTs: {FPMTs}, NLs: {NLs}, Es: {Es}, Time: {e1}')
        avg_time += e1
        print(e1)
        IGPath2 = savefile + '.edges' #sys.argv[1]
        lbPath2 = savefile + '.node_labels' #sys.argv[2]
        Separator = ','
        t = 2

        G = Graphdata(IGPath2, lbPath2, Separator).IG
        E = G.edge_list
        Ego = na.GreedyLP(G, t, E)
        Gc = na.Basic(Ge, t, Ego)
        # print(len(Gc.edge_list))
    else:
        e2 = time.time() - s2
        print(f'전처리시간 {e2}')
        
e2 = time.time() - s2
print(f'윈도우시간 {avg_time}')
print(f'종료시간 {e2}')
        

24344
edges 95.51 KB
node_labels 57.47 KB
pattern 408.0 B
pattern_nodes 113.55 KB
OutPut File Size: 266.93 KB bytes
edges 791.56 KB
node_labels 38.24 KB
InPut File Size: 829.8 KB bytes
Th : 24344, FPMTs: 22, NLs: 5914, Es: 7329, Time: 3.231712579727173
3.231712579727173
윈도우시간 3.231712579727173
종료시간 3.405076742172241


In [7]:
IGPath2 = savefile + '.edges' #sys.argv[1]
lbPath2 = savefile + '.node_labels' #sys.argv[2]
Separator = ','

G = Graphdata(IGPath2, lbPath2, Separator).IG
print(len(G.edge_list))

print(G.edge_list)

83
[['P1-0', '83'], ['P1-0', 'P0-6'], ['P1-0', 'P0-10'], ['P1-0', 'P0-1'], ['P1-0', 'P0-2'], ['P1-0', 'P0-5'], ['P1-0', '62'], ['P1-0', 'P0-23'], ['54', '36'], ['54', '27'], ['54', '18'], ['54', '45'], ['54', 'P0-24'], ['P0-0', '15'], ['P0-0', '51'], ['P0-0', 'P0-24'], ['P0-0', 'P0-9'], ['P0-0', 'P0-3'], ['P0-0', 'P0-4'], ['P0-1', 'P0-2'], ['P0-1', '71'], ['24', 'P0-7'], ['24', '23'], ['24', 'P0-9'], ['24', 'P0-4'], ['15', 'P0-3'], ['15', '14'], ['15', 'P0-13'], ['5', 'P0-17'], ['5', 'P0-23'], ['5', '6'], ['5', 'P0-6'], ['P0-2', 'P0-17'], ['P0-3', '33'], ['P0-3', '51'], ['P0-3', 'P0-14'], ['P0-4', '6'], ['P0-4', 'P0-16'], ['P0-4', '42'], ['32', 'P1-2'], ['32', 'P0-20'], ['32', 'P1-1'], ['32', '33'], ['P0-5', '83'], ['P0-5', '85'], ['P0-6', '41'], ['P0-7', 'P0-25'], ['P0-7', '42'], ['P1-1', '62'], ['P1-1', 'P0-15'], ['P1-1', '14'], ['P0-8', 'P1-2'], ['P0-8', '90'], ['P0-8', '92'], ['6', 'P0-9'], ['P0-9', 'P0-21'], ['P0-10', '57'], ['P1-2', '50'], ['P1-2', '90'], ['36', 'P0-22'], ['36', 

In [3]:
G = Graphdata(IGPath2, lbPath2, Separator).IG
E = G.edge_list
Gc = na.Basic(Ge, t, E)
print(len(Gc.edge_list))



77


In [91]:
fileName = 'PROTEINS-full'
IGPath = './dataset/' + fileName + '/' + fileName + '.edges' #sys.argv[1]
lbPath = './dataset/' + fileName + '/' + fileName + '.node_labels' #sys.argv[2]


def convert_size(size_bytes):
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    return "%s %s" % (s, size_name[i])

input_file = [IGPath, lbPath]
file_size1 = 0
for ip in input_file:
    file_size1 = file_size1 + os.path.getsize(ip)
    print(ip.split('.')[2], convert_size(os.path.getsize(ip)))
print('InPut File Size:', convert_size(file_size1), 'bytes')
e1 = time.time() - s1
print(f'Th : {Th1}, FPMTs: {FPMTs}, NLs: {NLs}, Es: {Es}, Time: {e1}')

edges 3.13 MB
node_labels 256.94 KB
InPut File Size: 3.39 MB bytes
Th : 31786, FPMTs: 100, NLs: 108, Es: 128, Time: 44.75308656692505
